In [79]:
import random
import numpy as np

In [80]:


# Course, Theory/Lab, Section, Section-Strength, Professor, First-lecture-day,

# First- lecture-timeslot, First-lecture-room, First-lecture-room-size, Second-
# lecture-day, Second-lecture-timeslot, Second-lecture-room, Second-lecture-
# room-size


# Define the range of values for each attribute
num_courses = 12
course_range = range(1, 100)
theory_lab_range = [1, 2]  # 1 for theory, 2 for lab
section_range = range(1, 5)
section_strength_range = range(20, 120)
professor_range = range(1, 100)
lecture_day_range = range(1, 6)  # 1-5 for Monday to Friday

theory_timeslot_range = range(1, 5)  # 1-4 for time slots
lab_timeslot_range = range(5, 9)  # 5-8 for time slots


lecture_room_range = range(1, 11)  # 1-10 for room numbers
lecture_room_size_range = [60,120]

def generate_random_data():
    # Random data
    courses = []
    for _ in range(num_courses):
        course = [
            random.choice(course_range),
            random.choice(theory_lab_range),
            random.choice(section_range),
            random.choice(section_strength_range),
            random.choice(professor_range),
            random.choice(lecture_day_range),
            random.choice(theory_timeslot_range),
            random.choice(lecture_room_range),
            random.choice(lecture_room_size_range),

        ]
        #for second day
        second_day = [
             #for second day
            random.choice(lecture_day_range),
            random.choice(theory_timeslot_range),
            course[7],
            random.choice(lecture_room_size_range),
        ]
        
        if course[1] == 2:  # for labs
            course[6] = random.choice(lab_timeslot_range)
            second_day = [
             #for second day
            course[5],
            course[6]+1,
            course[7],
            course[8],
        ]
        
        
        courses.append(course + second_day)
       
        
    
    return courses



In [81]:

#generating binary encoding for each course

def int_to_binary(value, num_bits):
    return format(value, '0' + str(num_bits) + 'b')

#generating chromosome
def generate_chromosome():
    chromosome = ""
    courses = generate_random_data()
    for course in courses:
        for attribute in course:
            chromosome += int_to_binary(attribute, 8)
    
    return chromosome

In [82]:
# convert binary chromosome back to courses array
def binary_to_courses(chromosome):
    courses = []
    num_attributes = 13  #each course has 13 attributes
    
    for i in range(0, len(chromosome), num_attributes * 8):
        course_data = []
        for j in range(num_attributes):
            start_index = i + j * 8
            end_index = start_index + 8
            attribute_binary = chromosome[start_index:end_index]
            attribute_value = int(attribute_binary, 2)
            course_data.append(attribute_value)
        courses.append(course_data)
    
    return courses

### Fitness Function

In [83]:
def calculate_fitness(chromosome):
  #decoding chromosome
    courses = binary_to_courses(chromosome)
 
    
    fitness = 0
    
    #calculating fitness based on hard constraints
     
    professor_count = [] #list to check each professor  is teaching only one class at a time
    room_count = [] #list to check each room is assigned to only one section at a time
    professor_courses = {} #dictionary to check each professor is teaching only 3 courses
    section_courses = {} #dictionary to check each section has no more than 5 courses
    for course in courses:
      #extracting details of each course
      
      course_name = course[0]
      theory_lab = course[1]
      section = course[2]
      section_strength = course[3]
      professor = course[4]
      first_lecture_day = course[5]
      first_lecture_timeslot = course[6]
      first_lecture_room = course[7]
      first_lecture_room_size = course[8]
      second_lecture_day = course[9]
      second_lecture_timeslot = course[10]
      second_lecture_room = course[11]
      second_lecture_room_size = course[12]
      
      
      #Classes can only be scheduled in free classrooms.
      
      if first_lecture_room==0 or second_lecture_room==0:
        fitness -= 1
        
      
      #A classroom should be big enough to accommodate the section. There should be two categories of classrooms: classroom (60) and large hall (120).
      if section_strength > first_lecture_room_size or section_strength > second_lecture_room_size:
        fitness -= 1
      
      
      #A professor should not be assigned two different lectures at the same time.
      if (professor, first_lecture_day, first_lecture_timeslot) in professor_count:
        fitness -= 1
      elif (professor, second_lecture_day, second_lecture_timeslot) in professor_count:
        fitness -= 1
      else:
        professor_count.append((professor, first_lecture_day, first_lecture_timeslot))
        professor_count.append((professor, second_lecture_day, second_lecture_timeslot))
      
      
      #A room cannot be assigned for two different sections at the same time.
      if (first_lecture_day, first_lecture_timeslot, first_lecture_room) in room_count:
        fitness -= 1
      elif (second_lecture_day, second_lecture_timeslot, second_lecture_room) in room_count:
        fitness -= 1
      else:
        room_count.append((first_lecture_day, first_lecture_timeslot, first_lecture_room))
        room_count.append((second_lecture_day, second_lecture_timeslot, second_lecture_room))
      
      
      #No professor can teach more than 3 courses.
      if professor in professor_courses:
        if course_name not in professor_courses[professor]:
          
          if len(professor_courses[professor]) >= 3:
            fitness -= 1
          else:
              professor_courses[professor].append(course_name)
      else:
        professor_courses[professor] = [course_name]
      
      
      #No section can have more than 5 courses in a semester.
         
      if section in section_courses:
          if course_name not in section_courses[section]:
              if len(section_courses[section]) >= 5:
                  fitness -= 1
              else:
                  section_courses[section].append(course_name)
      else:
          section_courses[section] = [course_name]
      
      #Each course would have two lectures per week not on the same or adjacent days.
      if abs(first_lecture_day - second_lecture_day) <= 1:
        fitness -= 1
      
      #Lab lectures should be conducted in two consecutive slots.
      if theory_lab == 2 and abs(first_lecture_timeslot - second_lecture_timeslot) != 1: #lab has two slots on same day
        fitness -= 1
       
        
      #A lab can scheduled only one class in a week
      if theory_lab == 2 and first_lecture_day != second_lecture_day:
        fitness -= 1
      
      
      #15 mins breaks allowed between consecutive classes to ensure that there is sufficient time for transitions between classes.
       # Not Implemented
      
      
      #calculating on soft constraints
      
      
      #All the theory classes should be taught in the morning session and all the lab sessions should be done in the afternoon session.
      if theory_lab == 2 and (first_lecture_timeslot < 5 or second_lecture_timeslot < 5):
        fitness -= 1
        
      elif theory_lab == 1 and (first_lecture_timeslot > 4 or second_lecture_timeslot > 4):
        fitness -= 1
        
      #A class should be held in the same classroom across the whole week.
      if first_lecture_room != second_lecture_room:
        fitness -= 1
    
    return fitness
    


In [84]:
def print_timetable(chromosome):
    courses = binary_to_courses(chromosome)
    print("course", "l/t", "section", "sec_strength", "prof", "day1", "timeslot1","room1","room1size","day2"
       ,"timeslot2","room2","room2size")
    for course in courses:
         joined =  "\t".join(map(str,course))
         print(joined)         
    print(f"\nBinary encoded chromosome\n{chromosome}")

### Initial population

In [85]:


def generate_initial_population(size):
    timetables = []

    # Generate 10 chromosomes and their fitnesses
    for _ in range(size):
        chromosome = generate_chromosome()
        fitness = calculate_fitness(chromosome)

        timetables.append((chromosome,fitness)) 


    return timetables


### Selection

In [86]:


def selection(sorted_timetables):
    
    selected_timetables = sorted_timetables[:2]
    return selected_timetables



### Crossover

In [87]:
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1)-2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return child1,child2



### Mutation

In [88]:
def Mutatation(chromosome, mutation_rate):
    # Define the range of values for each attribute
    num_courses = 10
    course_range = range(1, 100)
    theory_lab_range = [1, 2]  # 1 for theory, 2 for lab
    section_range = range(1, 4)
    section_strength_range = range(20, 120)
    professor_range = range(1, 100)
    lecture_day_range = range(1, 6)  # 1-5 for Monday to Friday
    lecture_timeslot_range = range(1, 9)  # 1-8 for time slots
    lecture_room_range = range(1, 11)  # 1-10 for room numbers
    lecture_room_size_range = [60,120]
    
    replacement_list = [[x for x in range(1, 100)], [1, 2], [x for x in range(1, 4)], [x for x in range(20, 120)], 
                        [x for x in range(1, 100)], [x for x in range(1, 6)], [x for x in range(1, 9)], 
                        [x for x in range(1, 11)], [60, 120],
                        [x for x in range(1, 6)], [x for x in range(1, 9)], [x for x in range(1, 11)], [60, 120]]
    
    
    timetable = binary_to_courses(chromosome)
    for i in range(len(timetable)):
        for j in range(len(timetable[i])):
            if random.random() < mutation_rate:
                timetable[i][j] = random.choice(replacement_list[j])
    
    # Convert timetable back to binary chromosome
    chromosome = ""
    for course in timetable:
        for attribute in course:
            chromosome += int_to_binary(attribute, 8)
    
    return chromosome  


### Genetic Algorithm

In [89]:
def Genetic_algorithm(generations):
    best_fitness = -999999
    best_chromosome = ""
    
    timetables = generate_initial_population(10)
    
    for generation in range(generations):
        print(f"=====================Generation {generation}=====================")
        
        timetables.sort(key=lambda x: x[1], reverse=True)

        selected_timetables = selection(timetables)

        child1,child2 = single_point_crossover(selected_timetables[0][0], selected_timetables[1][0])
        fitness_child1 = calculate_fitness(child1)
        fitness_child2 = calculate_fitness(child2)
        print(f"Fitness of child1 after crossover: {fitness_child1}")
        print(f"Fitness of child2 after crossover: {fitness_child2}")

        child1 = Mutatation(child1, 0.1)
        child2 = Mutatation(child2, 0.1)

        fitness_child1 = calculate_fitness(child1)
        fitness_child2 = calculate_fitness(child2)

        print(f"Fitness of Mutated child1: {fitness_child1}")
        print(f"Fitness of Mutated child2: {fitness_child2}")
        
        if fitness_child1 > best_fitness:
            best_fitness = fitness_child1
            best_chromosome = child1
        if fitness_child2 > best_fitness:
            best_fitness = fitness_child2
            best_chromosome = child2
        
        timetables = selected_timetables + [(child1, fitness_child1), (child2, fitness_child2)]
    print(f"===================== Result =====================")
    print(f"Best fitness: {best_fitness}")
    return best_chromosome
    

result = Genetic_algorithm(10)

print_timetable(result)


=====================Generation 0=====================
Fitness of child1 after crossover: -12
Fitness of child2 after crossover: -13
Fitness of Mutated child1: -20
Fitness of Mutated child2: -24
=====================Generation 1=====================
Fitness of child1 after crossover: -14
Fitness of child2 after crossover: -10
Fitness of Mutated child1: -22
Fitness of Mutated child2: -16
=====================Generation 2=====================
Fitness of child1 after crossover: -13
Fitness of child2 after crossover: -11
Fitness of Mutated child1: -18
Fitness of Mutated child2: -15
=====================Generation 3=====================
Fitness of child1 after crossover: -14
Fitness of child2 after crossover: -9
Fitness of Mutated child1: -16
Fitness of Mutated child2: -11
=====================Generation 4=====================
Fitness of child1 after crossover: -15
Fitness of child2 after crossover: -11
Fitness of Mutated child1: -22
Fitness of Mutated child2: -14
=====================Gener